In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
import torch
from acctrack.tools.model_inference import ModelLoader, TorchModelInference
from acctrack.io.pyg_data_reader import TrackGraphDataReader
from acctrack.tools.utils_graph import build_edges
import faiss

/media/DataOcean/miniconda3/envs/gnn4itk/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


uproot is not installed. AthenaRawRootReader will not be available.


In [10]:
model_path = "/media/DataOcean/projects/tracking/itk/For2023Paper/TrainedModels/graph_construction-MetricLearning-v1.pt"
data_path = "/home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store"

r_max, k_max = 0.1, 1000

In [4]:
model_infer = TorchModelInference("metric_learning-nonoise.yaml", model_path, ".")

Training: Total 1 events in directory: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/trainset
Validation: Total 1 events in directory: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/valset
Test: Total 1 events in directory: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/testset


## Embedding

In [5]:
data = model_infer.data_reader_training.read(0)

Reading file: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/trainset/event000005367-graph.pyg


In [6]:
node_features = model_infer.config["node_features"]
node_scales = torch.Tensor(model_infer.config["node_scales"])
input_data = torch.stack([data[x] for x in node_features], dim=-1).float()
features = input_data / node_scales

In [7]:
features[0]

tensor([ 0.0342,  0.9269, -0.0883,  1.0000,  0.5000, -1.1036,  0.1477,  0.1000,
         0.0500,  0.1500,  1.1036, -0.6479,  0.4164,  0.9828])

In [8]:
embedding = model_infer.model_reader.predict(features)

In [5]:
embedding = model_infer.inference(0)

Reading file: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/trainset/event000005367-graph.pyg


In [6]:
embedding[0], embedding.shape

tensor([-0.0908, -0.0284, -0.2994, -0.1605, -0.1913,  0.0260,  0.1929, -0.0615,
        -0.0207, -0.0559,  0.2816,  0.2985,  0.0521, -0.2373,  0.0643, -0.0638,
        -0.1706,  0.4789,  0.0141, -0.1035, -0.0988,  0.2270,  0.0684,  0.0094,
         0.2849,  0.0885, -0.1747, -0.0492,  0.0630, -0.0150,  0.0177, -0.3104],
       grad_fn=<SelectBackward0>)

In [17]:
edge_index = build_edges(embedding, embedding, r_max=r_max, k_max=k_max, backend=None)

In [19]:
print(edge_index.shape)

torch.Size([2, 20651722])


In [11]:
edge_index = build_edges(embedding, embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU")

: 

: 

In [ ]:
print(edge_index.shape)

Help on function knn in module faiss.extra_wrappers:

knn(xq, xb, k, metric=1)
    Compute the k nearest neighbors of a vector without constructing an index
    
    
    Parameters
    ----------
    xq : array_like
        Query vectors, shape (nq, d) where d is appropriate for the index.
        `dtype` must be float32.
    xb : array_like
        Database vectors, shape (nb, d) where d is appropriate for the index.
        `dtype` must be float32.
    k : int
        Number of nearest neighbors.
    distance_type : MetricType, optional
        distance measure to use (either METRIC_L2 or METRIC_INNER_PRODUCT)
    
    Returns
    -------
    D : array_like
        Distances of the nearest neighbors, shape (nq, k)
    I : array_like
        Labels of the nearest neighbors, shape (nq, k)



In [18]:
data = model_infer.data_reader_training.read(0)

Reading file: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/trainset/event000005367-graph.pyg


In [4]:
model_loader = ModelLoader(model_path)

In [5]:
model_loader.load()

In [6]:
model_loader.model

RecursiveScriptModule(
  original_name=MetricLearning
  (network): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=LayerNorm)
    (2): RecursiveScriptModule(original_name=ReLU)
    (3): RecursiveScriptModule(original_name=Linear)
    (4): RecursiveScriptModule(original_name=LayerNorm)
    (5): RecursiveScriptModule(original_name=ReLU)
    (6): RecursiveScriptModule(original_name=Linear)
    (7): RecursiveScriptModule(original_name=LayerNorm)
    (8): RecursiveScriptModule(original_name=ReLU)
    (9): RecursiveScriptModule(original_name=Linear)
    (10): RecursiveScriptModule(original_name=LayerNorm)
    (11): RecursiveScriptModule(original_name=ReLU)
    (12): RecursiveScriptModule(original_name=Linear)
  )
)

In [10]:
data_path = "/home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store"
data_path_train = data_path + "/trainset/"
data_reader = TrackGraphDataReader(data_path_train)


BaseTrackDataReader: Total 1 events in directory: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/trainset


In [11]:
data = data_reader.read(0)

Reading file: /home/xju/projects/tracking/itk/For2023Paper/2023_uncorr_feature_store/trainset/event000005367-graph.pyg


In [12]:
data

Data(lengthDir1_1=[349243], localDir1_2=[349243], x=[349243], cluster_z_2=[349243], norm_z_1=[349243], lengthDir2_1=[349243], loc_phi_2=[349243], loc_eta_1=[349243], eta_angle_2=[349243], count_2=[349243], glob_phi_1=[349243], r=[349243], cluster_y_1=[349243], localDir1_1=[349243], norm_y_2=[349243], norm_x_2=[349243], localDir2_2=[349243], charge_count_2=[349243], hit_id=[349243], count_1=[349243], loc_eta_2=[349243], lengthDir0_2=[349243], cluster_x_2=[349243], lengthDir0_1=[349243], phi_angle_2=[349243], lengthDir1_2=[349243], glob_eta_2=[349243], phi=[349243], glob_eta_1=[349243], loc_phi_1=[349243], eta_angle_1=[349243], cluster_x_1=[349243], y=[349243], localDir0_2=[349243], eta=[349243], norm_z_2=[349243], phi_angle_1=[349243], lengthDir2_2=[349243], cluster_z_1=[349243], cluster_y_2=[349243], norm_y_1=[349243], localDir0_1=[349243], region=[349243], localDir2_1=[349243], glob_phi_2=[349243], norm_x_1=[349243], charge_count_1=[349243], z=[349243], track_edges=[2, 136240], pt=[13

In [13]:
data['y'].shape

torch.Size([349243])

In [14]:
data['track_edges'].shape

torch.Size([2, 136240])